In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import numpy as np
from torchtext.data.utils import get_tokenizer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
data = pd.read_csv("data/reddit_vm.csv")
data.head(10)

,title,score,id,url,comms_num,created,body,timestamp
0,Health Canada approves AstraZeneca COVID-19 va...,7,lt74vw,https://www.canadaforums.ca/2021/02/health-can...,0,1.614400e+09,NaN,2021-02-27 06:33:45
1,COVID-19 in Canada: 'Vaccination passports' a ...,2,lsh0ij,https://www.canadaforums.ca/2021/02/covid-19-i...,1,1.614316e+09,NaN,2021-02-26 07:11:07
2,Coronavirus variants could fuel Canada's third...,6,lohlle,https://www.canadaforums.ca/2021/02/coronaviru...,0,1.613887e+09,NaN,2021-02-21 07:50:08
3,Canadian government to extend COVID-19 emergen...,1,lnptv8,https://www.canadaforums.ca/2021/02/canadian-g...,0,1.613796e+09,NaN,2021-02-20 06:35:13
4,Canada: Pfizer is 'extremely committed' to mee...,6,lkslm6,https://www.canadaforums.ca/2021/02/canada-pfi...,0,1.613468e+09,NaN,2021-02-16 11:36:28
5,Canada: Oxford-AstraZeneca vaccine approval ex...,5,lftbji,https://www.canadaforums.ca/2021/02/canada-oxf...,0,1.612869e+09,NaN,2021-02-09 13:17:11
6,Comment,1,ej9x066,NaN,0,1.553474e+09,Your OP. It's not a myth. Only one vaccine con...,2019-03-25 02:34:53
7,Fuck you anti-vaxxing retards,10,g6jkhp,https://www.reddit.com/r/VaccineMyths/comments...,8,1.587663e+09,https://youtu.be/zBkVCpbNnkU,2020-04-23 20:23:42
8,Comment,0,fofa0yy,NaN,0,1.587760e+09,Because Anti-Vaxxers have no sense,2020-04-24 23:19:50
9,Comment,0,ej9xuaf,NaN,0,1.553475e+09,"What do you mean by ""your OP"". I am fairly new...",2019-03-25 02:45:21


In [20]:
data.head(1).body

0    NaN
Name: body, dtype: object

In [30]:
data_full = data.dropna(subset = ["body"])
data_full

,title,score,id,url,comms_num,created,body,timestamp
6,Comment,1,ej9x066,NaN,0,1.553474e+09,Your OP. It's not a myth. Only one vaccine con...,2019-03-25 02:34:53
7,Fuck you anti-vaxxing retards,10,g6jkhp,https://www.reddit.com/r/VaccineMyths/comments...,8,1.587663e+09,https://youtu.be/zBkVCpbNnkU,2020-04-23 20:23:42
8,Comment,0,fofa0yy,NaN,0,1.587760e+09,Because Anti-Vaxxers have no sense,2020-04-24 23:19:50
9,Comment,0,ej9xuaf,NaN,0,1.553475e+09,"What do you mean by ""your OP"". I am fairly new...",2019-03-25 02:45:21
10,Comment,1,ej9x2qr,NaN,0,1.553474e+09,"When they say there's no thimerasol, they mean...",2019-03-25 02:35:47
...,...,...,...,...,...,...,...,...
1597,Comment,2,elrv9ua,NaN,0,1.556223e+09,Lol. My comprehension is comprehensive.\r\nYou...,2019-04-25 23:11:10
1598,Comment,1,elrud7d,NaN,0,1.556223e+09,"If you have trouble finding it, that's one thi...",2019-04-25 23:02:04
1599,Comment,0,elrtjl5,NaN,0,1.556222e+09,Is there music playing where you are because y...,2019-04-25 22:53:50
1600,Comment,1,elrrt5l,NaN,0,1.556221e+09,How long does it take ethyl mercury to metabol...,2019-04-25 22:35:39


In [33]:
descriptions = data_full.body.to_list()
descriptions[:5]

["Your OP. It's not a myth. Only one vaccine contains it and you can get it without it. So your OP is pointless flex. ",
 'https://youtu.be/zBkVCpbNnkU',
 'Because Anti-Vaxxers have no sense',
 'What do you mean by "your OP". I am fairly new to reddit.',
 "When they say there's no thimerasol, they mean in the childhood schedule. That IS a fact. "]

### Looking at word frequencies

In [49]:
tokenizer = get_tokenizer("basic_english")
threshold = 1000
desc_joined = ''.join(descriptions)
tokens = tokenizer(desc_joined)

freq_dist = FreqDist(tokens)
freq_dist
common_tokens = [token for token in tokens if freq_dist[token] > threshold]
print(common_tokens)

['.', 'it', "'", 'a', '.', 'it', 'and', 'you', 'it', 'it', '.', 'is', '.', '.', 'you', '.', 'i', 'to', '.', "'", ',', 'in', 'the', '.', 'that', 'is', 'a', '.', 'the', 'you', "'", 'is', 'in', 'to', 'the', '.', 'of', 'it', '.', 'you', "'", 'a', '.', 'you', "'", 'to', 'it', 'i', "'", 'that', ',', 'the', '.', 'you', 'the', '.', 'the', '.', 'to', ',', 'it', "'", 'of', 'and', 'in', '.', 'that', "'", '.', 'i', "'", 'to', "'", "'", '.', '.', '.', '.', 'i', "'", 'in', '.', 'the', ',', 'i', ',', 'is', '.', '.', ',', ',', 'the', 'is', '.', 'i', "'", 'is', '.', 'i', '.', 'you', "'", '.', ',', 'i', "'", 'you', "'", 'the', '.', "'", 'you', "'", 'to', '.', '.', '.', 'you', "'", 'the', 'you', 'in', '.', 'i', "'", 'that', 'you', ',', 'you', '.', ',', 'and', '.', 'and', 'you', "'", ',', 'you', 'of', '.', 'is', '.', 'of', 'to', 'you', 'in', 'and', ',', 'that', 'you', '.', 'it', "'", 'to', '.', 'to', 'is', 'a', 'of', "'", 'of', 'to', '.', 'that', "'", 'it', 'is', 'a', '.', 'and', 'that', "'", 'to', '.', '

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

# Initialize the tokenizer and stemmer
tokenizer = get_tokenizer("basic_english")
stemmer = PorterStemmer() 

# Complete the function to preprocess sentences
def preprocess_sentences(sentences):
    processed_sentences = []
    for sentence in sentences:
        sentence = sentence.lower()
        tokens = tokenizer(sentence)
        tokens = [token for token in tokens if token not in stop_words]
        tokens = [stemmer.stem(token) for token in tokens]
        processed_sentences.append(' '.join(tokens))
    return processed_sentences


processed_text = preprocess_sentences(descriptions)
print(processed_text[:5]) 


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/scottpitcher/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


<1226x7917 sparse matrix of type '<class 'numpy.float64'>'
	with 49039 stored elements in Compressed Sparse Row format>